# [Hate Speech Identification Shared Task](https://multihate.github.io/): Subtask 1A at [BLP Workshop](https://blp-workshop.github.io/) @IJCNLP-AACL 2025

This shared task is designed to identify the type of hate, its severity, and the targeted group from social media content. The goal is to develop robust systems that advance research in this area.

In this subtask, given a Bangla text collected from YouTube comments, categorize whether it contains abusive, sexism, religious hate, political hate, profane, or none.

### Downloading dataset from github

In [1]:
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1B/blp25_hatespeech_subtask_1B_train.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1B/blp25_hatespeech_subtask_1B_dev.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1B/blp25_hatespeech_subtask_1B_test.tsv

--2025-09-11 23:18:47--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1B/blp25_hatespeech_subtask_1B_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8005567 (7.6M) [text/plain]
Saving to: ‘blp25_hatespeech_subtask_1B_train.tsv’

blp25_hatespeech_su 100%[===================>]   7.63M  --.-KB/s    in 0.09s   

2025-09-11 23:18:48 (85.3 MB/s) - ‘blp25_hatespeech_subtask_1B_train.tsv’ saved [8005567/8005567]

--2025-09-11 23:18:48--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1B/blp25_hatespeech_subtask_1B_dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.co

### installing required libraries.
 - transformers
 - datasets
 - evaluate
 - accelerate

In [2]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install --upgrade accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00


#### importing required libraries and setting up logger

In [3]:
import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import datasets
import evaluate
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
import torch

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version


logger = logging.getLogger(__name__)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

### Defining the training, validation, and test data

In [4]:
train_file = 'blp25_hatespeech_subtask_1B_train.tsv'
validation_file = 'blp25_hatespeech_subtask_1B_dev.tsv'
test_file = 'blp25_hatespeech_subtask_1B_test.tsv'

### Disable wandb

In [5]:
import os
os.environ["WANDB_DISABLED"] = "true"

### Setting up the training parameters

In [6]:
# model to load elsewhere: "ai4bharat/IndicBERTv2-MLM-only"
from transformers import TrainingArguments

training_args = TrainingArguments(
    # === keep directory name unchanged ===
    output_dir="./distilBERT_m/",
    overwrite_output_dir=True,

    # === core training ===
    learning_rate=2e-5,
    num_train_epochs=3,                 # same as before
    per_device_train_batch_size=16,     # unchanged
    per_device_eval_batch_size=16,      # unchanged
    gradient_accumulation_steps=1,      # unchanged

    # === saving & checkpointing ===
    save_strategy="epoch",              # save checkpoints each epoch
    save_total_limit=2,                 # keep last 2 checkpoints

    # === optimization recipe ===
    lr_scheduler_type="linear",
    warmup_ratio=0.1,                  # unchanged; works fine for IndicBERT v2
    weight_decay=0.01,

    # === misc / perf ===
    remove_unused_columns=False,
    report_to=None,                     # disable TensorBoard/W&B logging
    seed=42,
    fp16=True,                          # if GPU supports (Ampere+)

    # === distributed (if running with DDP) ===
    local_rank=1,
    ddp_find_unused_parameters=False,
)

# keep your variables
max_train_samples = None
max_eval_samples = None
max_predict_samples = None

# sequence & batch
max_seq_length = 512   # unchanged
batch_size = 16        # unchanged

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [7]:
transformers.utils.logging.set_verbosity_info()

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f" distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=False,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=Inter

#### Defining the Model

In [8]:
model_name = 'ai4bharat/IndicBERTv2-MLM-only'

#### setting the random seed

In [9]:
set_seed(training_args.seed)

#### Loading data files

In [10]:
l2id = {'None': 0, 'Society': 1, 'Organization': 2, 'Community': 3, 'Individual': 4}
train_df = pd.read_csv(train_file, sep='\t')
# print(train_df['label'])
train_df['label'] = train_df['label'].map(l2id).fillna(0).astype(int)
train_df = Dataset.from_pandas(train_df)
validation_df = pd.read_csv(validation_file, sep='\t')
validation_df['label'] = validation_df['label'].map(l2id).fillna(0).astype(int)
validation_df = Dataset.from_pandas(validation_df)
test_df = pd.read_csv(test_file, sep='\t')
#test_df['label'] = test_df['label'].map(l2id)
test_df = Dataset.from_pandas(test_df)

data_files = {"train": train_df, "validation": validation_df, "test": test_df}
for key in data_files.keys():
    logger.info(f"loading a local file for {key}")
raw_datasets = DatasetDict(
    {"train": train_df, "validation": validation_df, "test": test_df}
)

INFO:__main__:loading a local file for train
INFO:__main__:loading a local file for validation
INFO:__main__:loading a local file for test


In [11]:
len(test_df['id'])

10200

##### Extracting number of unique labels

In [12]:
# Labels
label_list = raw_datasets["train"].unique("label")
print(label_list)
label_list.sort()  # sort the labels for determine
num_labels = len(label_list)

[0, 4, 1, 3, 2]


### Loading Pretrained Configuration, Tokenizer and Model

In [13]:
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=num_labels,
    finetuning_task=None,
    cache_dir=None,
    revision="main",
    use_auth_token=None,
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=None,
    use_fast=True,
    revision="main",
    use_auth_token=None,
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    from_tf=bool(".ckpt" in model_name),
    config=config,
    cache_dir=None,
    revision="main",
    use_auth_token=None,
    ignore_mismatched_sizes=False,
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/639 [00:00<?, ?B/s]

[INFO|configuration_utils.py:765] 2025-09-11 23:20:21,465 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--ai4bharat--IndicBERTv2-MLM-only/snapshots/8598f13fe52443bc3fc054fcd665944560145b5c/config.json
[INFO|configuration_utils.py:839] 2025-09-11 23:20:21,471 >> Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "dtype": "float32",
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12

tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.75M [00:00<?, ?B/s]

...

[INFO|tokenization_utils_base.py:2068] 2025-09-11 23:20:23,081 >> loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--ai4bharat--IndicBERTv2-MLM-only/snapshots/8598f13fe52443bc3fc054fcd665944560145b5c/tokenizer.json
[INFO|tokenization_utils_base.py:2068] 2025-09-11 23:20:23,084 >> loading file tokenizer.model from cache at None
[INFO|tokenization_utils_base.py:2068] 2025-09-11 23:20:23,085 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2068] 2025-09-11 23:20:23,086 >> loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--ai4bharat--IndicBERTv2-MLM-only/snapshots/8598f13fe52443bc3fc054fcd665944560145b5c/special_tokens_map.json
[INFO|tokenization_utils_base.py:2068] 2025-09-11 23:20:23,087 >> loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--ai4bharat--IndicBERTv2-MLM-only/snapshots/8598f13fe52443bc3fc054fcd665944560145b5c/tokenizer_config.json
[INFO|tok

pytorch_model.bin:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

[INFO|modeling_utils.py:1280] 2025-09-11 23:20:46,129 >> loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--ai4bharat--IndicBERTv2-MLM-only/snapshots/8598f13fe52443bc3fc054fcd665944560145b5c/pytorch_model.bin
[INFO|safetensors_conversion.py:61] 2025-09-11 23:20:46,229 >> Attempting to create safetensors variant
[INFO|safetensors_conversion.py:74] 2025-09-11 23:20:46,524 >> Safetensors PR exists


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

[INFO|modeling_utils.py:5711] 2025-09-11 23:20:49,502 >> Some weights of the model checkpoint at ai4bharat/IndicBERTv2-MLM-only were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification m

#### Preprocessing the raw_datasets

In [14]:
non_label_column_names = [name for name in raw_datasets["train"].column_names if name != "label"]
sentence1_key= non_label_column_names[1]

# Padding strategy
padding = "max_length"

# Some models have set the order of the labels to use, so let's make sure we do use it.
label_to_id = None
if (model.config.label2id != PretrainedConfig(num_labels=num_labels).label2id):
    # Some have all caps in their config, some don't.
    label_name_to_id = {k.lower(): v for k, v in model.config.label2id.items()}
    if sorted(label_name_to_id.keys()) == sorted(label_list):
        label_to_id = {i: int(label_name_to_id[label_list[i]]) for i in range(num_labels)}
    else:
        logger.warning(
            "Your model seems to have been trained with labels, but they don't match the dataset: ",
            f"model labels: {sorted(label_name_to_id.keys())}, dataset labels: {sorted(label_list)}."
            "\nIgnoring the model labels as a result.",)

if label_to_id is not None:
    model.config.label2id = label_to_id
    model.config.id2label = {id: label for label, id in config.label2id.items()}

if 128 > tokenizer.model_max_length:
    logger.warning(
        f"The max_seq_length passed ({128}) is larger than the maximum length for the"
        f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}.")
max_seq_length = min(128, tokenizer.model_max_length)

def preprocess_function(examples):
    # Tokenize the texts
    args = (
        (examples[sentence1_key],))
    result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)

    # Map labels to IDs (not necessary for GLUE tasks)
    if label_to_id is not None and "label" in examples:
        result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
    return result
raw_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    load_from_cache_file=True,
    desc="Running tokenizer on dataset",
)


Running tokenizer on dataset:   0%|          | 0/35522 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/2512 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/10200 [00:00<?, ? examples/s]

#### Finalize the training data for training the model

In [15]:
if "train" not in raw_datasets:
    raise ValueError("requires a train dataset")
train_dataset = raw_datasets["train"]
if max_train_samples is not None:
    max_train_samples_n = min(len(train_dataset), max_train_samples)
    train_dataset = train_dataset.select(range(max_train_samples_n))

In [16]:
train_dataset

Dataset({
    features: ['id', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 35522
})

#### Finalize the development/evaluation data for evaluating the model

In [17]:
if "validation" not in raw_datasets:
    raise ValueError("requires a validation dataset")
eval_dataset = raw_datasets["validation"]
if max_eval_samples is not None:
    max_eval_samples_n = min(len(eval_dataset), max_eval_samples)
    eval_dataset = eval_dataset.select(range(max_eval_samples_n))

#### Finalize the test data for predicting the unseen test data using the model

In [18]:
if "test" not in raw_datasets and "test_matched" not in raw_datasets:
    raise ValueError("requires a test dataset")
predict_dataset = raw_datasets["test"]
if max_predict_samples is not None:
    max_predict_samples_n = min(len(predict_dataset), max_predict_samples)
    predict_dataset = predict_dataset.select(range(max_predict_samples_n))

#### Log a few random samples from the training set

In [19]:
for index in random.sample(range(len(train_dataset)), 3):
    logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

INFO:__main__:Sample 7296 of the training set: {'id': 660, 'text': 'সরকারের দায়িত্বপ্রাপ্ত সংস্থা সকল বাণিজ্যিক ভবন গুলোকে বছরে অন্তত একবার পরিদর্শন করে রিপোর্ট প্রদান করবেন একটা বাণিজ্যিক ভবনে নিরাপত্তার সকল ব্যবস্থা থাকা উচিত কিন্তু এই সব সম্ভবের দেশে কিছুই করা হয় না এই দেশে মানুষের জীবনের কোন দাম নেই তাই কেউ কোনকিছুর তদারকিরও দরকার মনে করে না', 'label': 0, 'input_ids': [1, 28632, 175326, 35870, 20349, 66323, 60609, 95425, 15745, 51774, 43670, 48399, 84122, 16066, 41763, 30178, 29204, 21480, 66323, 113089, 73628, 20349, 26322, 32615, 27882, 17537, 16182, 18289, 29053, 15534, 29172, 36046, 16899, 16628, 16251, 16182, 29172, 27709, 42202, 17146, 33574, 20098, 21397, 23487, 17146, 150002, 244917, 9196, 9168, 36157, 21616, 16066, 16251, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], 'token_type_ids': [0, 0, 0, 0

#### Get the metric function `accuracy`

In [20]:
metric = evaluate.load("accuracy")

#### Predictions and label_ids field and has to return a dictionary string to float.

In [21]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)

    acc  = accuracy_score(p.label_ids, preds)
    f1   = f1_score(p.label_ids, preds, average="micro")   # এখানে micro
    prec = precision_score(p.label_ids, preds, average="micro")
    rec  = recall_score(p.label_ids, preds, average="micro")

    return {
        "accuracy": acc,
        "f1": f1,
        "precision": prec,
        "recall": rec,
    }

#### Data Collator

In [22]:
data_collator = default_data_collator

#### Initialize our Trainer

In [23]:
train_dataset = train_dataset.remove_columns("id")
eval_dataset = eval_dataset.remove_columns("id")

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipython-input-1383425655.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
[INFO|trainer.py:757] 2025-09-11 23:21:15,768 >> Using auto half precision backend


#### Training our model

In [25]:
train_result = trainer.train()
metrics = train_result.metrics
max_train_samples = (
    max_train_samples if max_train_samples is not None else len(train_dataset)
)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))



[WARNING|trainer.py:985] 2025-09-11 23:21:15,784 >> The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 3}.
[INFO|trainer.py:2523] 2025-09-11 23:21:16,390 >> ***** Running training *****
[INFO|trainer.py:2524] 2025-09-11 23:21:16,391 >>   Num examples = 35,522
[INFO|trainer.py:2525] 2025-09-11 23:21:16,394 >>   Num Epochs = 3
[INFO|trainer.py:2526] 2025-09-11 23:21:16,396 >>   Instantaneous batch size per device = 16
[INFO|trainer.py:2529] 2025-09-11 23:21:16,399 >>   Total train batch size (w. parallel, distributed & accumulation) = 16
[INFO|trainer.py:2530] 2025-09-11 23:21:16,401 >>   Gradient Accumulation steps = 1
[INFO|trainer.py:2531] 2025-09-11 23:21:16,402 >>   Total optimization steps = 6,663
[INFO|trainer.py:2532] 2025-09-11 23:21:16,405 >>   Number of trainable parameters = 278,045,189


Step,Training Loss
500,1.189200
1000,0.849600
1500,0.786800
2000,0.767900
2500,0.710300
3000,0.651200
3500,0.680300
4000,0.656300
4500,0.639200
5000,0.548600


[INFO|trainer.py:4289] 2025-09-11 23:27:35,537 >> Saving model checkpoint to ./distilBERT_m/checkpoint-2221
[INFO|configuration_utils.py:491] 2025-09-11 23:27:35,540 >> Configuration saved in ./distilBERT_m/checkpoint-2221/config.json
[INFO|modeling_utils.py:4297] 2025-09-11 23:27:51,829 >> Model weights saved in ./distilBERT_m/checkpoint-2221/model.safetensors
[INFO|tokenization_utils_base.py:2563] 2025-09-11 23:27:51,832 >> tokenizer config file saved in ./distilBERT_m/checkpoint-2221/tokenizer_config.json
[INFO|tokenization_utils_base.py:2572] 2025-09-11 23:27:51,833 >> Special tokens file saved in ./distilBERT_m/checkpoint-2221/special_tokens_map.json
[INFO|trainer.py:4289] 2025-09-11 23:34:29,436 >> Saving model checkpoint to ./distilBERT_m/checkpoint-4442
[INFO|configuration_utils.py:491] 2025-09-11 23:34:29,440 >> Configuration saved in ./distilBERT_m/checkpoint-4442/config.json
[INFO|modeling_utils.py:4297] 2025-09-11 23:34:50,934 >> Model weights saved in ./distilBERT_m/checkp

#### Saving the tokenizer too for easy upload

In [26]:
trainer.save_model()
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

[INFO|trainer.py:4289] 2025-09-11 23:42:11,727 >> Saving model checkpoint to ./distilBERT_m/
[INFO|configuration_utils.py:491] 2025-09-11 23:42:11,730 >> Configuration saved in ./distilBERT_m/config.json
[INFO|modeling_utils.py:4297] 2025-09-11 23:42:27,400 >> Model weights saved in ./distilBERT_m/model.safetensors
[INFO|tokenization_utils_base.py:2563] 2025-09-11 23:42:27,403 >> tokenizer config file saved in ./distilBERT_m/tokenizer_config.json
[INFO|tokenization_utils_base.py:2572] 2025-09-11 23:42:27,405 >> Special tokens file saved in ./distilBERT_m/special_tokens_map.json


***** train metrics *****
  epoch                    =        3.0
  total_flos               =  6528442GF
  train_loss               =      0.702
  train_runtime            = 0:20:55.30
  train_samples            =      35522
  train_samples_per_second =     84.892
  train_steps_per_second   =      5.308


#### Evaluating our model on validation/development data

In [27]:
logger.info("*** Evaluate ***")

metrics = trainer.evaluate(eval_dataset=eval_dataset)

max_eval_samples = (
    max_eval_samples if max_eval_samples is not None else len(eval_dataset)
)
metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))
#print(metrics)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

INFO:__main__:*** Evaluate ***
[INFO|trainer.py:4623] 2025-09-11 23:42:27,494 >> 
***** Running Evaluation *****
[INFO|trainer.py:4625] 2025-09-11 23:42:27,494 >>   Num examples = 2512
[INFO|trainer.py:4628] 2025-09-11 23:42:27,495 >>   Batch size = 16


***** eval metrics *****
  epoch                   =        3.0
  eval_accuracy           =     0.7293
  eval_f1                 =     0.7293
  eval_loss               =     0.7434
  eval_precision          =     0.7293
  eval_recall             =     0.7293
  eval_runtime            = 0:00:05.09
  eval_samples            =       2512
  eval_samples_per_second =     492.82
  eval_steps_per_second   =     30.801


#### epchos - 03 --> 72.73
#### epchos - 04 --> 72.37

### Predecting the test data

In [28]:
id2l = {v: k for k, v in l2id.items()}
logger.info("*** Predict ***")
#predict_dataset = predict_dataset.remove_columns("label")
ids = predict_dataset['id']
predict_dataset = predict_dataset.remove_columns("id")
predictions = trainer.predict(predict_dataset, metric_key_prefix="predict").predictions
predictions = np.argmax(predictions, axis=1)
output_predict_file = "subtask_1B.tsv"
if trainer.is_world_process_zero():
    with open(output_predict_file, "w") as writer:
        logger.info(f"***** Predict results *****")
        writer.write("id\tlabel\tmodel\n")
        for index, item in enumerate(predictions):
            item = label_list[item]
            item = id2l[item]
            writer.write(f"{ids[index]}\t{item}\t{model_name}\n")

INFO:__main__:*** Predict ***
[INFO|trainer.py:4623] 2025-09-11 23:42:32,612 >> 
***** Running Prediction *****
[INFO|trainer.py:4625] 2025-09-11 23:42:32,613 >>   Num examples = 10200
[INFO|trainer.py:4628] 2025-09-11 23:42:32,614 >>   Batch size = 16


INFO:__main__:***** Predict results *****


In [29]:
ids[0]

12764

#### Saving the model into card

In [30]:
kwargs = {"finetuned_from": model_name, "tasks": "text-classification"}
trainer.create_model_card(**kwargs)

[INFO|modelcard.py:456] 2025-09-11 23:42:56,923 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.7292993630573248}, {'name': 'F1', 'type': 'f1', 'value': 0.7292993630573248}, {'name': 'Precision', 'type': 'precision', 'value': 0.7292993630573248}, {'name': 'Recall', 'type': 'recall', 'value': 0.7292993630573248}]}


In [31]:
!zip subtask_1B.zip subtask_1B.tsv

  adding: subtask_1B.tsv (deflated 89%)
